In [3]:
import os
import argparse
import importlib
from datetime import datetime
import pytz
import re
import numpy as np

from classifier_lewidi import *
from params_lewidi import *
from DataManager import *


In [4]:
filepath1 = '../data/ConvAbuse_dataset/'
filepath2 = '../data/ArMIS_dataset/'
filepath3 = '../data/HS-Brexit_dataset/'
filepath4 = '../data/MD-Agreement_dataset/'

In [5]:
%reload_ext autoreload
%autoreload 2


data = DataManager(filepath1, filepath2, filepath3, filepath4)

I'm in with the conv_train dataset
I'm in with the conv_dev dataset
I'm in with the conv_test dataset
I'm in with the ar_train dataset
I'm in with the ar_dev dataset
I'm in with the ar_test dataset
I'm in with the br_train dataset
I'm in with the br_dev dataset
I'm in with the br_test dataset
I'm in with the md_train dataset
I'm in with the md_dev dataset
I'm in with the md_test dataset


In [6]:
def_params = params_lewidi()


In [8]:

%reload_ext autoreload
%autoreload 2

key_train = "ar_train"
key_dev = "ar_dev"
key_test =  "ar_test"

if key_train == "ar_train":
    def_params.ar_dat = 1
elif key_train == "md_train":
    def_params.task = "single"

annotators = list(sorted(set(itertools.chain.from_iterable(data.dataset_groups[key_train]['annotators'].str.findall("\w+")))))
    
model = AbuseClassifier(data.dataset_groups[key_train], data.dataset_groups[key_dev], data.dataset_groups[key_test] , annotators=annotators, params=def_params)



Train data shape after majority voting (657, 11)
Train data shape after majority voting (141, 11)
[('batch_size', 64), ('learning_rate', 1e-07), ('max_len', 128), ('num_epochs', 1), ('random_state', 9999), ('task', 'multi_task'), ('batch_weight', None), ('sort_by', None), ('predict', 'label'), ('mc_passes', 10), ('ar_dat', 1)]


In [14]:
%reload_ext autoreload
%autoreload 2


# score, results = model.CV()
results = model.CV()



1 hello


In [ ]:
comp_metrics = results
comp_metrics["soft_label_0"] = np.nan
comp_metrics["soft_label_1"] = np.nan
comp_metrics["hard_label"] = np.nan

for ind, row in results.iterrows():
    
    count_zero, count_one, totalcount = 0,0,0
    for element in row:
        if np.isnan(element):
            continue
        elif element == 0:
            count_zero+= 1 
            totalcount +=1
        else:
            count_one +=1
            totalcount +=1   
                  
    soft_zero, soft_one = count_zero / float(totalcount) , count_one / float(totalcount)

    comp_metrics["soft_label_0"][ind] = soft_zero
    comp_metrics["soft_label_1"][ind] = soft_one

    if soft_zero > soft_one:
        comp_metrics["hard_label"][ind] = 0
    elif soft_zero < soft_one:
        comp_metrics["hard_label"][ind] = 1
    else:
        comp_metrics["hard_label"][ind] = random.randrange(0, 2)
        
comp_metrics = comp_metrics.loc[:,["hard_label","soft_label_0", "soft_label_1"]]
comp_metrics["hard_label"]= pd.to_numeric(comp_metrics["hard_label"], downcast="integer")

In [ ]:
results_filepath = "../data/results"
if key_test == "conv_test":
    # "MD-Agreement_results.tsv", "ArMIS_results.tsv", "ConvAbuse_results.tsv", "HS-Brexit_results.tsv"
    result_dir = os.path.join(results_filepath + "ConvAbuse_results.tsv")
    os.makedirs("../data/results", exist_ok=True)  
    comp_metrics.to_csv("../data/results/ConvAbuse_results.tsv", sep="\t", header = False,index=False)

In [ ]:
print("done")
x=0
# pacific = pytz.timezone('US/Pacific')
# sa_time = datetime.now(pacific)
# name_time = sa_time.strftime('%m%d%y-%H:%M')
# score["time"] = name_time
# print(score)

# score_dir = os.path.join(def_params.source_dir, "results", "GHC", "classification.csv")
# result_dir = os.path.join(def_params.source_dir, "results", "GHC", name_time + "_" + def_params.task + ".csv")
# if os.path.exists(score_dir):
#     pd.DataFrame.from_records([score]).to_csv(score_dir, header=False,  index=False, mode="a")
# else:
#     pd.DataFrame.from_records([score]).to_csv(score_dir, index=False)

#     pd.DataFrame.from_dict(results).to_csv(result_dir, index=False)


done
